实现tranformer模块

In [ ]:
def run_transformer_block(
    d_model: int,
    num_heads: int,
    d_ff: int,
    max_seq_len: int,
    theta: float,
    weights: dict[str, Tensor],
    in_features: Float[Tensor, " batch sequence_length d_model"],
) -> Float[Tensor, " batch sequence_length d_model"]:
    """
    给定预归一化Transformer块的权重和输入特征，
    返回在输入特征上运行Transformer块的输出。

    此函数应使用RoPE。
    根据你的实现，你可能只需要将相关参数传递给
    TransformerBlock构造函数，或者你可能需要初始化自己的RoPE
    类并传递它。

    参数:
        d_model (int): Transformer块输入的维度。
        num_heads (int): 多头注意力中使用的头数。`d_model`必须
            能被`num_heads`整除。
        d_ff (int): 前馈内层的维度。
        max_seq_len (int): 如果你的实现预缓存的最大序列长度。
        theta (float): RoPE参数。
        weights (dict[str, Tensor]):
            我们参考实现的状态字典。
            此字典的键包括:
            - `attn.q_proj.weight`
                所有`num_heads`个注意力头的查询投影。
                形状为(d_model, d_model)。
                行按形状为(num_heads, d_k)的矩阵排序，
                所以`attn.q_proj.weight == torch.cat([q_heads.0.weight, ..., q_heads.N.weight], dim=0)`。
            - `attn.k_proj.weight`
                所有`num_heads`个注意力头的键投影。
                形状为(d_model, d_model)。
                行按形状为(num_heads, d_k)的矩阵排序，
                所以`attn.k_proj.weight == torch.cat([k_heads.0.weight, ..., k_heads.N.weight], dim=0)`。
            - `attn.v_proj.weight`
                所有`num_heads`个注意力头的值投影。
                形状为(d_model, d_model)。
                行按形状为(num_heads, d_v)的矩阵排序，
                所以`attn.v_proj.weight == torch.cat([v_heads.0.weight, ..., v_heads.N.weight], dim=0)`。
            - `attn.output_proj.weight`
                多头自注意力输出投影的权重
                形状为(d_model, d_model)。
            - `ln1.weight`
                变换器块中应用的第一个RMSNorm的仿射变换权重。
                形状为(d_model,)。
            - `ffn.w1.weight`
                FFN中第一个线性变换的权重。
                形状为(d_model, d_ff)。
            - `ffn.w2.weight`
                FFN中第二个线性变换的权重。
                形状为(d_ff, d_model)。
            - `ffn.w3.weight`
                FFN中第三个线性变换的权重。
                形状为(d_model, d_ff)。
            - `ln2.weight`
                变换器块中应用的第二个RMSNorm的仿射变换权重。
                形状为(d_model,)。
        in_features (Float[Tensor, "batch sequence_length d_model"]):
            运行实现的张量。

    返回:
        Float[Tensor, "batch sequence_length d_model"] 在使用RoPE时
        在输入特征上运行Transformer块的输出张量。
    """

In [ ]:

class Transformer_block(nn.Module):
    """
    Transformer Block (预归一化版本)
    
    这是现代 Transformer 架构的基本构建块，采用预归一化设计，
    比原始 Transformer 的后归一化更稳定、更容易训练。
    
    架构组成:
    1. Pre-RMSNorm + Multi-Head Self-Attention + Residual Connection
    2. Pre-RMSNorm + Feed-Forward Network (SwiGLU) + Residual Connection
    
    相比后归一化的优势:
    - 梯度流更稳定
    - 训练收敛更快
    - 支持更深的网络
    """
    
    def __init__(self, 
                 d_model: int, 
                 num_heads: int, 
                 d_ff: int, 
                 max_seq_len: int, 
                 theta: float | None = None):
        """
        初始化 Transformer Block
        
        参数:
            d_model (int): 模型的特征维度 (如 512, 768, 1024)
                          必须能被 num_heads 整除
            num_heads (int): 多头注意力的头数 (如 8, 12, 16)
            d_ff (int): 前馈网络的隐藏层维度
                       通常是 d_model 的 4 倍 (如 d_model=512 -> d_ff=2048)
            max_seq_len (int): 支持的最大序列长度
                              用于 RoPE 位置编码的预计算
            theta (float | None): RoPE 的基础角度参数
                                 如果为 None，则不使用位置编码
                                 如果不为 None (如 10000.0)，则启用 RoPE
        """
        super().__init__()
        
        # 保存关键参数用于调试和后续使用
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.theta = theta
        
        # 条件性初始化注意力模块
        if theta is not None:
            # 创建 RoPE 位置编码器
            # d_k = d_model // num_heads：每个头的维度
            pos_encode = RotaryPositionalEmbedding(
                theta=theta, 
                d_k=d_model // num_heads,  # 注意：RoPE 应用于每个头
                max_seq_len=max_seq_len
            )
            
            # 带位置编码的多头自注意力
            self.attn = Multihead_self_attention(
                d_model=d_model, 
                num_heads=num_heads, 
                pos_encode=pos_encode, 
                theta=theta
            )
        else:
            # 不带位置编码的多头自注意力
            self.attn = Multihead_self_attention(
                d_model=d_model, 
                num_heads=num_heads
            )
        
        # Layer Normalization 层
        # 使用 RMSNorm 替代传统 LayerNorm，计算更高效
        self.rmsn_1 = RMSNorm(d_model=d_model, eps=1e-5)  # 注意力前的归一化
        self.rmsn_2 = RMSNorm(d_model=d_model, eps=1e-5)  # FFN 前的归一化
        
        # Feed-Forward Network (FFN)
        # 使用 SwiGLU 替代传统的 ReLU FFN，表达能力更强
        # SwiGLU: SwiGLU(x) = (SiLU(xW1) ⊙ xW3)W2
        self.pw_ffn = SwiGLU(d_model=d_model, d_ff=d_ff)
        
        # 备选：传统的 SiLU FFN (已注释)
        # self.pw_ffn = SiLU(d_model=d_model, d_ff=d_ff)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Transformer Block 前向传播 (预归一化版本)
        
        计算流程:
        1. x -> RMSNorm -> Multi-Head Attention -> + x (第一个残差连接)
        2. 结果 -> RMSNorm -> FFN -> + 结果 (第二个残差连接)
        
        参数:
            x (torch.Tensor): 输入张量
                             形状: (batch_size, sequence_length, d_model)
        
        返回:
            torch.Tensor: 输出张量，形状与输入相同
                         (batch_size, sequence_length, d_model)
        """
        
        # === 第一个子层：Multi-Head Self-Attention ===
        
        # 1. 预归一化：在注意力计算前先归一化
        # 这是与原始 Transformer 的关键差异 (原始是后归一化)
        normalized_x = self.rmsn_1(x)
        
        # 2. 多头自注意力计算
        # 输入和输出形状都是 (batch_size, sequence_length, d_model)
        attn_output = self.attn(normalized_x)
        
        # 3. 第一个残差连接
        # 将注意力输出与原始输入相加，保持梯度流
        # 这使得网络可以学习恒等映射，有助于训练深层网络
        out1 = x + attn_output
        
        # === 第二个子层：Feed-Forward Network ===
        
        # 4. 预归一化：在 FFN 计算前归一化
        normalized_out1 = self.rmsn_2(out1)
        
        # 5. 前馈网络计算
        # SwiGLU: 先上投影到 d_ff 维度，应用门控激活，再下投影回 d_model
        # 形状变化: (batch, seq, d_model) -> (batch, seq, d_ff) -> (batch, seq, d_model)
        ffn_output = self.pw_ffn(normalized_out1)
        
        # 6. 第二个残差连接
        # 将 FFN 输出与第一个子层的输出相加
        out = out1 + ffn_output
        
        return out

因果语言模型

In [ ]:

def run_transformer_lm(
    vocab_size: int,
    context_length: int,
    d_model: int,
    num_layers: int,
    num_heads: int,
    d_ff: int,
    rope_theta: float,
    weights: dict[str, Tensor],
    in_indices: Int[Tensor, " batch_size sequence_length"],
) -> Float[Tensor, " batch_size sequence_length vocab_size"]:
    """
    给定Transformer语言模型的权重和输入索引，
    返回在输入索引上运行前向传播的输出。

    此函数应使用RoPE。

    参数:
        vocab_size (int): 要预测的输出词汇表中的唯一项目数。
        context_length (int): 一次处理的最大token数。
        d_model (int): 模型嵌入和子层输出的维度。
        num_layers (int): 要使用的Transformer层数。
        num_heads (int): 多头注意力中使用的头数。`d_model`必须
            能被`num_heads`整除。
        d_ff (int): 前馈内层的维度(第3.3节)。
        rope_theta (float): RoPE Θ参数。
        weights (dict[str, Tensor]):
            我们参考实现的状态字典。{num_layers}指的是
            `0`到`num_layers - 1`之间的整数(层索引)。
            此字典的键包括:
            - `token_embeddings.weight`
                Token嵌入矩阵。形状为(vocab_size, d_model)。
            - `layers.{num_layers}.attn.q_proj.weight`
                所有`num_heads`个注意力头的查询投影。
                形状为(num_heads * (d_model / num_heads), d_model)。
                行按形状为(num_heads, d_k)的矩阵排序，
                所以`attn.q_proj.weight == torch.cat([q_heads.0.weight, ..., q_heads.N.weight], dim=0)`。
            - `layers.{num_layers}.attn.k_proj.weight`
                所有`num_heads`个注意力头的键投影。
                形状为(num_heads * (d_model / num_heads), d_model)。
                行按形状为(num_heads, d_k)的矩阵排序，
                所以`attn.k_proj.weight == torch.cat([k_heads.0.weight, ..., k_heads.N.weight], dim=0)`。
            - `layers.{num_layers}.attn.v_proj.weight`
                所有`num_heads`个注意力头的值投影。
                形状为(num_heads * (d_model / num_heads), d_model)。
                行按形状为(num_heads, d_v)的矩阵排序，
                所以`attn.v_proj.weight == torch.cat([v_heads.0.weight, ..., v_heads.N.weight], dim=0)`。
            - `layers.{num_layers}.attn.output_proj.weight`
                多头自注意力输出投影的权重
                形状为((d_model / num_heads) * num_heads, d_model)。
            - `layers.{num_layers}.ln1.weight`
                变换器块中应用的第一个RMSNorm的仿射变换权重。
                形状为(d_model,)。
            - `layers.{num_layers}.ffn.w1.weight`
                FFN中第一个线性变换的权重。
                形状为(d_model, d_ff)。
            - `layers.{num_layers}.ffn.w2.weight`
                FFN中第二个线性变换的权重。
                形状为(d_ff, d_model)。
            - `layers.{num_layers}.ffn.w3.weight`
                FFN中第三个线性变换的权重。
                形状为(d_model, d_ff)。
            - `layers.{num_layers}.ln2.weight`
                变换器块中应用的第二个RMSNorm的仿射变换权重。
                形状为(d_model,)。
            - `ln_final.weight`
                应用于最终变换器块输出的RMSNorm的仿射变换权重。
                形状为(d_model, )。
            - `lm_head.weight`
                语言模型输出嵌入的权重。
                形状为(vocab_size, d_model)。
        in_indices (Int[Tensor, "batch_size sequence_length"]) 运行语言模型的输入索引张量。形状为(batch_size, sequence_length)，其中
            `sequence_length`最多为`context_length`。

    返回:
        Float[Tensor, "batch_size sequence_length vocab_size"]: 每个token的预测未归一化
        下一词分布的张量。
    """

In [ ]:
class Transformer_lm(nn.Module):
    def __init__(self, vocab_size:int, context_length:int, num_layers: int, d_model: int, num_heads: int, d_ff: int, rope_theta: float | None = None):
        super().__init__()
        self.context_length = context_length
        self.transformer = nn.ModuleDict(dict(
            token_emb = Embedding(num_embedding=vocab_size, embedding_dim=d_model),
            n_block = nn.ModuleList([Transformer_block(d_model=d_model, num_heads=num_heads, d_ff=d_ff, max_seq_len=context_length, theta=rope_theta) for _ in range(num_layers)]),
            rmsn_l = RMSNorm(d_model=d_model, eps=1e-5)
        ))
        self.linear_emb = Linear(d_model, vocab_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        tkemb = self.transformer.token_emb(x)
        for block in self.transformer.n_block:
            tkemb = block(tkemb)
        tkemb = self.transformer.rmsn_l(tkemb)
        out = self.linear_emb(tkemb)
        return out

    @torch.no_grad()
    def generate(self, x: torch.Tensor, max_gen_tokens: int, temperature: float = 1.0, top_p: int | None = None, eos_token_id: int | None = None):
        if x.dim() == 1:
            x = x.unsqueeze(0)
            
        original_sequence_length = x.size(-1)
        for _ in range(max_gen_tokens):
            x = x[:, -self.context_length :] if x.size(1) > self.context_length else x
            logits = self.forward(x)
            next_token_logits = logits[:, -1, :]
            temperature_scaled = next_token_logits / temperature
            if top_p:
                sorted_logits, sorted_indices = torch.sort(temperature_scaled, descending=True)
                sorted_probs = run_softmax(sorted_logits, dim=-1)
                cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
                sorted_mask = cumulative_probs > top_p
                sorted_mask[..., 1:] = sorted_mask[..., :-1].clone()
                sorted_mask[..., 0] = False
                mask = sorted_mask.scatter(1, sorted_indices, sorted_mask)
                temperature_scaled = temperature_scaled.masked_fill(mask, float("-inf"))
            probs = run_softmax(temperature_scaled, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            if eos_token_id is not None and next_token_id.item() == eos_token_id:
                break
            x = torch.cat((x, next_token_id), dim=-1)
        return x[:, original_sequence_length:]

    @classmethod
    def from_pretrained(cls, pretrained_path: str):
        with open(os.path.join(pretrained_path, "model_config.json")) as f:
            config = json.load(f)
        model = cls(**config)
        weights_path = os.path.join(pretrained_path, "model.pt")
        state_dict = torch.load(weights_path, weights_only=True)
        # Remove _orig_mod. prefix that comes from serializing a compiled model
        unwanted_prefix = "_orig_mod."
        for k, _ in list(state_dict.items()):
            if k.startswith(unwanted_prefix):
                state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
        model.load_state_dict(state_dict)
        return model

    def save_pretrained(self, pretrained_path: str):
        os.makedirs(pretrained_path, exist_ok=True)
        config = {
            "vocab_size": self.transformer["token_emb"].weight.size(0),
            "context_length": self.context_length,
            "num_layers": len(self.transformer["n_block"]),
            "d_model": self.transformer["token_emb"].weight.size(1),
            "num_heads": self.transformer["n_block"][0].num_heads,
            "d_ff": self.transformer["n_block"][0].pw_ffn.d_ff,
            "rope_theta": self.transformer["n_block"][0].theta
        }
        with open(Path(pretrained_path) / "model_config.json", "w") as f:
            json.dump(config, f, indent=4)
        torch.save(self.state_dict(), Path(pretrained_path) / "model.pt")
